### Goal: prepare the dataset to feed the neural network

In [1]:
import pandas as pd
import numpy as np

Import the dataset + the list of features we want to keep as inputs for the neural network

In [2]:
features = pd.read_csv("dataset_before_encoding.csv", delimiter = ',', low_memory = False)
keep_features =  pd.read_csv("feature_importance.csv", delimiter = ',', names = ["options", "importance"])

Selection of the features selected by Hugo (with decision trees)

In [19]:
important_features = features[keep_features['options']]
important_features['vmlinux'] = features['vmlinux']
important_features['cid'] = features['cid']
important_features.head()

C:\Users\llesoil\Documents\Anaconda3\envs\tensor_gpu\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\llesoil\Documents\Anaconda3\envs\tensor_gpu\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,DEBUG_INFO,RANDOMIZE_BASE,DEBUG_INFO_REDUCED,DEBUG_INFO_SPLIT,STRICT_MODULE_RWX,BLK_MQ_PCI,MODULES,UBSAN_SANITIZE_ALL,X86_NEED_RELOCS,KASAN,...,MFD_WM5110,GCC_PLUGIN_RANDSTRUCT,HWMON_VID,BT,REGULATOR_TPS65023,BMI160_I2C,DNOTIFY,VIDEO_TW9903,vmlinux,cid
0,n,y,n,n,y,n,y,n,y,y,...,n,n,y,n,m,m,y,y,50222120,30000
1,n,n,n,n,y,n,y,n,n,n,...,y,n,y,n,n,m,y,n,16660024,30001
2,n,n,n,n,n,n,n,n,n,n,...,n,n,y,n,y,n,y,n,43080856,30002
3,n,n,n,n,n,y,n,n,n,n,...,n,n,y,n,y,y,n,y,27261672,30003
4,n,n,n,n,n,n,n,n,n,n,...,y,y,y,y,n,y,y,y,58769440,30004


Data processing :
- keep configurations with id > 30000
- keep kernel_size (here vmlinux) positive
- fill na
- remove other sizes
- keep only the yes/no/module features

In [20]:
data = important_features.fillna('na')

remove_columns = []

col = data.columns
types = data.dtypes
for i in range(len(col)):
    name_col = col[i]
    actual_col = data[name_col]
    if len(actual_col.unique()) == 1:
        remove_columns.append(name_col)
    if types[i] != np.dtype('object'):
        if name_col not in remove_columns:
            remove_columns.append(name_col)
remove_columns.remove('vmlinux')

remove_lines = []

cid = np.array(data["cid"], dtype = int)
for index_row in range(len(cid)):
    if cid[index_row] < 30000:
        remove_lines.append(index_row)

vmlinux = np.array(data["vmlinux"], dtype = int)
for index_row in range(len(vmlinux)):
    if vmlinux[index_row] < 0:
        if index_row not in remove_lines:
            remove_lines.append(index_row)


data_final = data.drop(remove_columns, axis = 1).drop(remove_lines, axis = 0)
data_final.head()

,DEBUG_INFO,RANDOMIZE_BASE,DEBUG_INFO_REDUCED,DEBUG_INFO_SPLIT,STRICT_MODULE_RWX,BLK_MQ_PCI,MODULES,UBSAN_SANITIZE_ALL,X86_NEED_RELOCS,KASAN,...,NTFS_FS,MFD_WM5110,GCC_PLUGIN_RANDSTRUCT,HWMON_VID,BT,REGULATOR_TPS65023,BMI160_I2C,DNOTIFY,VIDEO_TW9903,vmlinux
0,n,y,n,n,y,n,y,n,y,y,...,n,n,n,y,n,m,m,y,y,50222120
1,n,n,n,n,y,n,y,n,n,n,...,n,y,n,y,n,n,m,y,n,16660024
2,n,n,n,n,n,n,n,n,n,n,...,y,n,n,y,n,y,n,y,n,43080856
3,n,n,n,n,n,y,n,n,n,n,...,n,n,n,y,n,y,y,n,y,27261672
4,n,n,n,n,n,n,n,n,n,n,...,n,y,y,y,y,n,y,y,y,58769440


In [22]:
data_final.shape

(65855, 1124)

For each 'good' feature, we just keep one dummy 'the option is activated' (yes) vs 'the option is not activated' (module/no)

In [23]:
burt_table = pd.get_dummies(data_final.drop(['vmlinux'], axis = 1))
burt_table.head()
burt_table['vmlinux'] = data_final['vmlinux']
#burt_table.to_csv("burt_important.csv")

names = burt_table.columns
drop = []
for n in names:
    if n[len(n)-1] != 'y':
        drop.append(n)
drop.remove('vmlinux')
burt_table_yes = burt_table.drop(drop, axis = 1)
# about 150 Mo
burt_table_yes.to_csv("burt_yes_important.csv")